<h1>Aplicaciones Redes Neuronales<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clasificación-Binaria" data-toc-modified-id="Clasificación-Binaria-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clasificación Binaria</a></span><ul class="toc-item"><li><span><a href="#Preprocesamiento-de-datos" data-toc-modified-id="Preprocesamiento-de-datos-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preprocesamiento de datos</a></span></li><li><span><a href="#Modelo-de-clasificación-con-una-red-neuronal" data-toc-modified-id="Modelo-de-clasificación-con-una-red-neuronal-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Modelo de clasificación con una red neuronal</a></span></li></ul></li><li><span><a href="#Clasificación-Multinomial" data-toc-modified-id="Clasificación-Multinomial-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Clasificación Multinomial</a></span></li><li><span><a href="#Regresión-Lineal" data-toc-modified-id="Regresión-Lineal-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Regresión Lineal</a></span></li><li><span><a href="#Conclusiones" data-toc-modified-id="Conclusiones-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conclusiones</a></span></li></ul></div>

Caso tomado de [Applied Deep Learning - Part 2: Real World Case Studies](https://towardsdatascience.com/applied-deep-learning-part-2-real-world-case-studies-1bb4b142a585#581e) de   https://towardsdatascience.com.


# Ejemplos reales de aplicación de redes neuronales

Para este módulo estudiaremos tres casos diferentes de aplicación de redes neuronales. El objetivo del ejercicio es que al final tengamos clara la forma en la cual se diseña una red neuronal para resolver un problema real, cada caso consiste en una situación distinta de aplicación que utilizan el mismo modelo. Iniciamos con una clasificación binaria, luego una clasificación multiclase y finalmente una regresión.

## Clasificación Binaria

Usaremos un conjunto de datos de Análisis de recursos humanos de Kaggle. La idea es predecir si un empleado dejará una compañía en función de diversas características, como la cantidad de proyectos en los que trabajó, el tiempo que pasó en la empresa, la última revisión de desempeño, el salario, entre otros. 

Contamos con un conjunto de datos de 15,000 registros y 9 variables, entre las que se destaca la variable "left" que será nuestra variable objetivo. Dicha variable admite valores 0 o 1 donde 1 significa que el empleado se ha ido.

### Preprocesamiento de datos

Siempre recuerde hacer un preprocesamiento de datos adecuado, la idea es responder las siguientes preguntas:

* Qué tipo de características tenemos: valoradas reales, categóricas o ambas?
* ¿Alguna de las funciones necesita normalización?
* ¿Tenemos valores nulos?
* ¿Cuál es la distribución de etiquetas? ¿Están las clases desequilibradas?
* ¿Existe una correlación entre las características?

Empecemos llamando las librerias importantes, creando algunas funciones útiles y finalmente cargando los datos:

In [0]:
#Paquetes
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

from __future__ import print_function
from datetime import datetime
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
import pandas_profiling as pdp

In [0]:
#Funciones útiles
def plot_decision_boundary(func, X, y, figsize=(9, 6)):
    amin, bmin = X.min(axis=0) - 0.1
    amax, bmax = X.max(axis=0) + 0.1
    hticks = np.linspace(amin, amax, 101)
    vticks = np.linspace(bmin, bmax, 101)
    
    aa, bb = np.meshgrid(hticks, vticks)
    ab = np.c_[aa.ravel(), bb.ravel()]
    c = func(ab)
    cc = c.reshape(aa.shape)

    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    
    fig, ax = plt.subplots(figsize=figsize)
    contour = plt.contourf(aa, bb, cc, cmap=cm, alpha=0.8)
    
    ax_c = fig.colorbar(contour)
    ax_c.set_label("$P(y = 1)$")
    ax_c.set_ticks([0, 0.25, 0.5, 0.75, 1])
    
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
    plt.xlim(amin, amax)
    plt.ylim(bmin, bmax)

def plot_multiclass_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))
    cmap = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    Z = model.predict_classes(np.c_[xx.ravel(), yy.ravel()], verbose=0)
    Z = Z.reshape(xx.shape)
    fig = plt.figure(figsize=(8, 8))
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    
def plot_data(X, y, figsize=None):
    if not figsize:
        figsize = (8, 6)
    plt.figure(figsize=figsize)
    plt.plot(X[y==0, 0], X[y==0, 1], 'or', alpha=0.5, label=0)
    plt.plot(X[y==1, 0], X[y==1, 1], 'ob', alpha=0.5, label=1)
    plt.xlim((min(X[:, 0])-0.1, max(X[:, 0])+0.1))
    plt.ylim((min(X[:, 1])-0.1, max(X[:, 1])+0.1))
    plt.legend()

def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

def plot_loss(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, historydf.values.max()))
    plt.title('Loss: %.3f' % history.history['loss'][-1])
    
def plot_confusion_matrix(model, X, y):
    y_pred = model.predict_classes(X, verbose=0)
    plt.figure(figsize=(8, 6))
    sns.heatmap(pd.DataFrame(confusion_matrix(y, y_pred)), annot=True, fmt='d', cmap='GnBu', alpha=0.8, vmin=0)

def plot_compare_histories(history_list, name_list, plot_accuracy=True):
    dflist = []
    min_epoch = len(history_list[0].epoch)
    losses = []
    for history in history_list:
        h = {key: val for key, val in history.history.items() if not key.startswith('val_')}
        dflist.append(pd.DataFrame(h, index=history.epoch))
        min_epoch = min(min_epoch, len(history.epoch))
        losses.append(h['loss'][-1])

    historydf = pd.concat(dflist, axis=1)

    metrics = dflist[0].columns
    idx = pd.MultiIndex.from_product([name_list, metrics], names=['model', 'metric'])
    historydf.columns = idx
    
    plt.figure(figsize=(6, 8))

    ax = plt.subplot(211)
    historydf.xs('loss', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
    plt.title("Training Loss: " + ' vs '.join([str(round(x, 3)) for x in losses]))
    
    if plot_accuracy:
        ax = plt.subplot(212)
        historydf.xs('acc', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
        plt.title("Accuracy")
        plt.xlabel("Epochs")
    
    plt.xlim(0, min_epoch-1)
    plt.tight_layout()
    
def make_sine_wave():
    c = 3
    num = 2400
    step = num/(c*4)
    np.random.seed(0)
    x0 = np.linspace(-c*np.pi, c*np.pi, num)
    x1 = np.sin(x0)
    noise = np.random.normal(0, 0.1, num) + 0.1
    noise = np.sign(x1) * np.abs(noise)
    x1  = x1 + noise
    x0 = x0 + (np.asarray(range(num)) / step) * 0.3
    X = np.column_stack((x0, x1))
    y = np.asarray([int((i/step)%2==1) for i in range(len(x0))])
    return X, y

def make_multiclass(N=500, D=2, K=3):
    """
    N: number of points per class
    D: dimensionality
    K: number of classes
    """
    np.random.seed(0)
    X = np.zeros((N*K, D))
    y = np.zeros(N*K)
    for j in range(K):
        ix = range(N*j, N*(j+1))
        # radius
        r = np.linspace(0.0,1,N)
        # theta
        t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = j
    fig = plt.figure(figsize=(6, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu, alpha=0.8)
    plt.xlim([-1,1])
    plt.ylim([-1,1])
    return X, y

In [0]:
#Datos
DF=pd.read_csv('HR.csv')
DF

Respondamos la primera pregunta, tipo de datos y resumen:

In [0]:
DF.info()

Una inspección rápida para saber si debemos estandarizar los datos:

In [0]:
DF.describe()

Veamos sus correlaciones:

In [0]:
plt.figure(figsize=(10, 8))
sns.heatmap(DF.corr(), annot=True, square=True, vmin=-1, vmax=1)

Ahora veamos la distribución de cada una de las variables para ver cuáles necesitan estandarización. Selccionamos para ello 

* average_monthly_hours,
* number_project y 
* time_spend_company. 

¿Por qué?

In [0]:
pdp.ProfileReport(DF)

In [0]:
df = DF.copy()

ss = StandardScaler()
scale_features = ['average_monthly_hours', 'number_project', 'time_spend_company']
df[scale_features] = ss.fit_transform(df[scale_features])
df


Ahora bien, estamos muy contentos con nuestra base estandarizada, sin embargo nos enfrentamos a otro problema, las variables categóricas. Recuerden que cuando se definió el perceptrón lo hicimos como un elemento matemático y como tal debe tener entradas numéricas, no obstante, como lo vimos en clases pasadas entendimos que una forma de volver numérica una variable categórica es usando variables dummies:

In [0]:
categorical_features = ['sales', 'salary']
df_cat = pd.get_dummies(df[categorical_features])
df = df.drop(categorical_features, axis=1)
df = pd.concat([df, df_cat], axis=1)
df.head()

Los datos están listos pero falta la partición de la base en entrenamiento y testeo, como tenemos una base generosa utilizaremos el 30% de los registros para hacer testeo:

In [0]:
X = df.drop('left', axis=1).values
y = df['left'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

### Modelo de clasificación con una red neuronal

Ahora vamos a crear un modelo para predecir la variable left, aprovecharemois el desorden para jugar un poco con la tasa de aprendizaje:

In [0]:
bin_clas = Sequential()
bin_clas.add(Dense(64, input_shape=(X_train.shape[1],), activation='tanh'))
bin_clas.add(Dense(16, activation='tanh'))
bin_clas.add(Dense(1, activation='sigmoid'))

bin_clas.compile(Adam(lr=0.1), 'binary_crossentropy', metrics=['accuracy'])

bin_hist = bin_clas.fit(X_train, y_train, epochs=100)

In [0]:
plot_loss_accuracy(bin_hist)

Ahora con una tasa de aprendizaje no tan complicada:

In [0]:
bin_clas = Sequential()
bin_clas.add(Dense(64, input_shape=(X_train.shape[1],), activation='tanh'))
bin_clas.add(Dense(32, input_shape=(X_train.shape[1],), activation='tanh'))
bin_clas.add(Dense(16, activation='tanh'))
bin_clas.add(Dense(1, activation='sigmoid'))

bin_clas.compile(Adam(lr=0.005), 'binary_crossentropy', metrics=['accuracy'])

bin_hist = bin_clas.fit(X_train, y_train, epochs=100)

In [0]:
plot_loss_accuracy(bin_hist)

Buena aproximación, grafiquemos la matriz de confusión:

In [0]:
y_pred = bin_clas.predict_classes(X_test, verbose=0)
print(classification_report(y_test, y_pred))
plot_confusion_matrix(bin_clas, X_test, y_test)

Veamos como nos fue en la de entrenamiento:

In [0]:
y_pred = bin_clas.predict_classes(X_train, verbose=0)
print(classification_report(y_train, y_pred))
plot_confusion_matrix(bin_clas, X_train, y_train)

## Clasificación Multinomial

Ahora trabajaremos con otra base [Car Evaluation DataBase](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) se derivó de un modelo de decisión jerárquico simple desarrollado originalmente para la demostración de DEX, M. Bohanec, V. Rajkovic: Sistema experto para la toma de decisiones. Sistemica 1 (1), págs. 145-157, 1990.). El modelo evalúa los coches de acuerdo con la siguiente estructura conceptual:

Aceptabilidad del coche CAR
* PRECIO precio total
        buying : precio de compra
        maint price: precio de mantenimiento
* Características técnicas TECH
* CONFORT:  comodidad
        doors:  número de puertas
        persons capacity:  personas para llevar
        lug_boot: el tamaño del maletero
        safety:  seguridad estimada del coche

La base de datos relaciona directamente el automóvil con los seis atributos de entrada: compra, mantenimiento, puertas, personas, maletero, seguridad.

Debido a la estructura de concepto subyacente conocida, esta base de datos puede ser particularmente útil para probar métodos de inducción constructiva y descubrimiento de estructuras.

Información de atributos:

Valores de clase:

unacc, acc, good, vgood

Atributos:

* buying: vhigh, high, med, low.
* maint: vhigh, high, med, low.
* doors: 2, 3, 4, 5 or more.
* persons: 2, 4, more.
* lug_boot: small, med, big.
* safety: low, med, high.



In [0]:
DF2=pd.read_csv('car.data',header=None)
DF2.columns=['buying','maint','doors','persons','lug_boot','safety','clas']
DF2

In [0]:
df=DF2.copy()
categorical_features=df.columns[:-1]
df_cat = pd.get_dummies(df[categorical_features])
df.drop(categorical_features, axis=1)
df = pd.concat([ df_cat,df['clas']], axis=1)
df

Como las variables de entrada son categóricas no hacemos ninguna estandarización, iniciamos planteando el modelo, en primer lugar adecuamos la bases:

In [0]:
X = np.asarray(df.values[:, :-1]).astype('float32')
y = np.asarray(pd.get_dummies(df['clas']).values).astype('float32')
print(X.shape, y.shape)

Ahora definimos entrenamiento y testeo:

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape,X_test.shape,y_test.shape)

Finalmente definimos un modelo:

In [0]:
X_train

In [0]:
mul_clas = Sequential()
mul_clas.add(Dense(32, input_shape=(X.shape[1],), activation='relu'))
mul_clas.add(Dense(16, activation='relu'))
mul_clas.add(Dense(8, activation='relu'))
mul_clas.add(Dense(4, activation='softmax'))
mul_clas.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

history = mul_clas.fit(X_train, y_train, epochs=16, verbose=0)
plot_loss_accuracy(history)

In [0]:
y_pred_class = mul_clas.predict_classes(X_test, verbose=0)
y_test_class = np.argmax(y_test, axis=1)
print(classification_report(y_test_class, y_pred_class))
plot_confusion_matrix(mul_clas, X_test, y_test_class)

## Regresión Lineal

Usando la sintáxis de las redes neuronales modelaremos una regresión lineal, utilizaremos los datos de Kaggle para el caso [House Sales in King County, USA](https://www.kaggle.com/harlfoxem/housesalesprediction).  

In [0]:
DF3=pd.read_csv('houses_price.csv',index_col='id')
DF3

Hagamos una rápida exploración:

In [0]:
pdp.ProfileReport(DF3)

Veamos la correlación entre variables:

In [0]:
plt.figure(figsize=(20, 24))
sns.heatmap(DF3.corr(), annot=True, vmin=-1, vmax=1)

Las características relacionadas con pies cuadrados definitivamente deben estandarizarse, ya que los valores varían en miles y las características como el código postal deben clasificarse.

También tenemos un nuevo tipo de preprocesamiento que realizar, *bucketization*. Por ejemplo, la característica que contiene el año en que se construyó la casa (yr_built), varía de 1900 a 2015. Ciertamente podemos clasificarla con cada año perteneciente a una categoría distinta, pero entonces sería bastante escasa. Obtendríamos más señal si agrupamos esta función sin perder mucha información. Por ejemplo, si usamos cubos de 10 años, los años entre [1950, 1959] se colapsarían juntos. Probablemente sería suficiente saber que esta casa fue construida en la década de 1950 en lugar de en 1958 exactamente.

Otras características que se beneficiarían de la *bucketization* son la latitud y la longitud de la casa. La coordenada exacta no importa tanto, podemos redondear la coordenada al kilómetro más cercano. De esta manera, los valores de las características serán más densos e informativos. No existe una regla fija y estricta sobre los rangos que se deben usar en la agrupación en segmentos, se deciden principalmente por prueba y error.

Una última transformación que debemos hacer es el precio de la casa, actualmente su valor oscila entre $\$ 75 K$ y $\$ 7.7 M$. Un modelo que intenta predecir a una escala y variación tan grande sería muy inestable. Así que también normalizamos eso.


In [0]:
df = DF3.copy()

# Estandarizaciones
ss = StandardScaler()
scale_features = ['bathrooms', 'bedrooms', 'grade', 'sqft_above', 
                  'sqft_basement', 'sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_lot15']
df[scale_features] = ss.fit_transform(df[scale_features])

# bucketization
bucketized_features = ['yr_built', 'yr_renovated', 'lat', 'long']

bins = range(1890, 2021, 10)
df['yr_built'] = pd.cut(df.yr_built, bins, labels=bins[:-1])

bins = list(range(1930, 2021, 10))
bins = [-10] + bins
df['yr_renovated'] = pd.cut(df.yr_renovated, bins, labels=bins[:-1])

bins = np.arange(47.00, 47.90, 0.05)
df['lat'] = pd.cut(df.lat, bins, labels=bins[:-1])

bins = np.arange(-122.60, -121.10, 0.05)
df['long'] = pd.cut(df.long, bins, labels=bins[:-1])

# Variables categóricas
df['date'] = [datetime.strptime(x, '%Y%m%dT000000').strftime('%Y-%m') for x in DF3['date'].values]
df['zipcode'] = df['zipcode'].astype('str')
categorical_features = ['zipcode', 'date']
categorical_features.extend(bucketized_features)
df_cat = pd.get_dummies(df[categorical_features])
df = df.drop(categorical_features, axis=1)
df = pd.concat([df, df_cat], axis=1)
df

Nos preparamos para hacer el modelo:

In [0]:
X = df.drop(['price'], axis=1).values
y = df['price'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Arreglo de outliers
factor = 5
y_train[np.abs(y_train - y_train.mean()) > (factor * y_train.std())] = y_train.mean() + factor*y_train.std()

# Estandarizar precio
ss_price = StandardScaler()
y_train = ss_price.fit_transform(y_train.reshape(-1, 1))
y_test = ss_price.transform(y_test.reshape(-1, 1))

El modelo lineal no tendrá función de activación y la función de pérdida será el Error cuadrático medio:

In [0]:
linr_model = Sequential()
linr_model.add(Dense(1, input_shape=(X.shape[1],)))

linr_model.compile('adam', 'mean_squared_error')

linr_history = linr_model.fit(X_train, y_train, epochs=30, verbose=0, validation_split=0.2)
plot_loss(linr_history)

In [0]:
linr_model.evaluate(X_test, y_test, verbose=0)

In [0]:
# coeficientes
linr_wdf = pd.DataFrame(linr_model.get_weights()[0].T, 
                      columns=df.drop(['price'], axis=1).columns).T.sort_values(0, ascending=False)
linr_wdf.columns = ['feature_weight']
linr_wdf


Finalmente construimos una Red Neuronal para la regresión. En los ejemplos anteriores, pasar de un modelo lineal a un modelo profundo solo implicaba agregar nuevas capas con funciones de activación no lineales. Será lo mismo esta vez también.

Agregamos nuevas capas con activación relu. La trama de pérdidas parece interesante ahora. La pérdida de error de entrenamiento todavía parece estar disminuyendo, pero el error de validación comienza a aumentar después de la quinta época. 

In [0]:
deep_model = Sequential()
deep_model.add(Dense(32, input_shape=(X.shape[1],), activation='relu'))
deep_model.add(Dense(16, activation='relu'))
deep_model.add(Dense(8, activation='relu'))
deep_model.add(Dense(1))

deep_model.compile('adam', 'mean_squared_error')

# early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
deep_history = deep_model.fit(X_train, y_train, epochs=30, verbose=0, validation_split=0.2)
#                               callbacks=[early_stop])
plot_loss(deep_history)

Claramente estamos sobreajustando. La ANN está memorizando los datos de entrenamiento y esto está reduciendo su capacidad de generalizar en el conjunto de validación.

La regularización es un método creado para evitar el sobreajuste, sin embargo, podemos pensar en uno más simple, parar el procesamiento cuando se empiece a sentir el cambio entre la medida de las funciones de pérdida:

In [0]:
# con early stopping
deep_model = Sequential()
deep_model.add(Dense(32, input_shape=(X.shape[1],), activation='relu'))
deep_model.add(Dense(16, activation='relu'))
deep_model.add(Dense(8, activation='relu'))
deep_model.add(Dense(1))

deep_model.compile('adam', 'mean_squared_error')

early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
deep_history = deep_model.fit(X_train, y_train, epochs=30, verbose=0, validation_split=0.2,
                              callbacks=[early_stop])
plot_loss(deep_history)

In [0]:
plot_compare_histories([linr_history, deep_history], ['Linear Reg', 'Deep ANN'], plot_accuracy=False)

In [0]:
def output_dollars(num):
    return '$'+str("{:,}".format(int(num)))

print('Average house price:', output_dollars(DF3['price'].mean()))

real_prices = ss_price.inverse_transform(y_test)

# print('Training set house price:', np.mean(ss_price.inverse_transform(y_train)))

train_prices = ss_price.inverse_transform(y_train)
dumb_prices = np.zeros(real_prices.shape)
dumb_prices.fill(train_prices.mean())
dumb_error = mean_absolute_error(real_prices, dumb_prices)
print('Dumb model error:', output_dollars(dumb_error))

linr_predictions = linr_model.predict(X_test).ravel()
linr_prices = ss_price.inverse_transform(linr_predictions)
linr_error = mean_absolute_error(real_prices, linr_prices)
print('Linear model error:', output_dollars(linr_error))

deep_predictions = deep_model.predict(X_test).ravel()
deep_prices = ss_price.inverse_transform(deep_predictions)
deep_error = mean_absolute_error(real_prices, deep_prices)
print('Deep model error:', output_dollars(deep_error))

tdf = pd.DataFrame([['Naive Model', output_dollars(dumb_error)], 
                    ['Linear Regression', output_dollars(linr_error)], 
                    ['Deep ANN', output_dollars(deep_error)]], 
                   columns=['Model', 'Price Error'])
tdf

## Conclusiones

Usamos las redes neuronales para tres tipos de problemas típicos del Machine Learning, evidentemente el ajuste del modelo a los datos del entrenamiento produce potro gran lio, el sobreajuste, veremos como enfrentarlo en otro ejercicio. 
